# GBDT 调参指南

## 初始化
 
 1. 使用默认参数。默认学习率-0.1
 2. 子采样（subsample）典型值-[0.5,0.8]
 
 
## 调参步骤

 1. 网格搜索-寻找最佳迭代次数？
 2. 网格搜索，寻找最佳最大深度（max_depth）以及 子叶最小分裂阈值（min_samples_split）（参与寻找，但是因为和其他参数关联，暂时不确定）
 3. 网格搜索-同时确认最小节点分裂阈值与 终点节点最小样本数（min_samples_leaf）
 4. 网格搜索-寻找最大特征数（max_features）
 5. 网格搜索-寻找最佳子采样比例（subsample）
 6. 已经确定最佳模型，开始减半步长，增加迭代次数，来提高泛化能力。不断尝试直到出现泛华能力下降（过拟合）

## 最终模型

```python
GradientBoostingClassifier(learning_rate=0.01, 
                           n_estimators=600,
                           max_depth=7, 
                           min_samples_leaf =60, 
                           min_samples_split =1200,
                           max_features=9, 
                           subsample=0.7, 
                           random_state=10)
```

# GBM调参指南 

## 初始化

 1. 最大特征数（max_ features），典型值-[0.3-0.4],或者 ‘sqrt'
 2. 子叶最小分裂阈值（min_ samples_ split），推荐总样本数的[0.5%,1%],不均等分类问题，请选择较小的数值，本处——500.
 3. 子采样，推荐 0.8
 4. 终点节点最小样本数（min_samples_leaf），凭感觉选择一个合适的数字，只要不会造成过拟合即可。因为不均等分类问题，奔出选择一个较小的值。50
 5. max_depth.典型值[5.10],此处选择5.
 
 
 ## 模型调参
 
 
 1. 固定学习率（0.1），网格搜索估测决策树数量
 2. 调节树参数(网格搜索）
     1. 调节max_depth和 num_samples_split
     2. 调节min_samples_leaf
     3. 调节max_features
 3. 调节子样本比例
 4. ，并降低learning rate，增加步长。确定最佳模型。
 
 
 ## 最终模型

```python
GradientBoostingClassifier(learning_rate=0.005, 
                           n_estimators=1500,
                           max_depth=9, 
                           min_samples_split=1200, 
                           min_samples_leaf=60, 
                           subsample=0.85, 
                           random_state=10, 
                           max_features=7,
                           warm_start=True)
```

# XGB 调参指南

## 基本介绍

XGBoost的作者把所有的参数分成了三类：

 1. 通用参数：宏观函数控制。
 2. Booster参数：控制每一步的booster(tree/regression)。
 3. 学习目标参数：控制训练目标的表现。
 
## 调参步骤

 1. 初始化预估决策树个数
 2. 调节树的参数
     1. 预估最大深度与终点节点最小样本占比的和
     2. 预估gamma参数
     3. 调整子采样与列采样比例
 3. 调整正则化系数
 4. 减少学习率，增加决策树数量以确定最佳模型。
 
## 初始化，预估决策树个数
 
 1. 学习率（eta），典型值[0.01，0.2]，这里选择0.1
 2. min_child_weight --- 终点节点最小样本占比 的 和 默认等于一，使用默认选择
 3. 最大深度（max_depth）x，此处选择-5，典型值[4，6]
 4. gamma，典型值[0.1,0.2]，此处默认零，之后需要调整。
 5. 子采样，列采样 = 0.8 ，典型值[0.5,0.9]
 6. scale_pos_weight,不平衡时加快瘦脸，此处选1
 7. 使用网格搜索，预估决策树个数，策略如下
 

## 对树的参数进行调节
### 预估最大深度（max_depth）与终点节点最小样本占比 的 和（min_child_weight）

第一步，初步估计
```python
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}

gsearch1 = GridSearchCV(estimator = XGBClassifier(
    learning_rate =0.1, 
    n_estimators=140, 
    max_depth=5,
    min_child_weight=1, 
    gamma=0, 
    subsample=0.8,             
    colsample_bytree=0.8,
    objective= 'binary:logistic', 
    nthread=4,     
    scale_pos_weight=1, 
    seed=27), 
                        param_grid = param_test1,     
                        scoring='roc_auc',
                        n_jobs=4,iid=False, 
                        cv=5)

gsearch1.fit(train[predictors],train[target])
```
第二步骤，减少步长预估，逐步寻找

```python
param_test2 = {
 'max_depth':[4,5,6],
 'min_child_weight':[4,5,6]
}
```

### gamma 参数调优

具体含义：
 - 在节点分裂时，只有分裂后损失函数的值下降了，才会分裂这个节点。Gamma指定了节点分裂所需的最小损失函数下降值。
 - 这个参数的值越大，算法越保守。这个参数的值和损失函数息息相关，所以是需要调整的。
 
代码：
```python
param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
```
### 调整子采样（subsample）与列采样 （colsample_bytree)
```python
# 第一次估计，步长0.1
param_test4 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}

# 第二次估计，步长0.05
param_test5 = {
 'subsample':[i/100.0 for i in range(75,90,5)],
 'colsample_bytree':[i/100.0 for i in range(75,90,5)]
}
```

## 调整正则化参数
gamma本身已经提过了一种有效的降低过拟合的方法，但是这里我们仍然进行正则化参数的调整。下面是alpha的网格搜索，lambda同理。
```python
# 第一次估计
param_test6 = {
 'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}
# 第二次估计
param_test7 = {
 'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05]
}
```

## 降低学习率，增加拟合次数，确定最终模型

## 存在的问题
### 参数估计
 1. 终点节点样本占比问题- min_ weight_ fraction_leaf？&& 终点节点样本数量问题 - max_ leaf_ nodes ？